In [1]:
from src.dataset import load_insurance
from src.counterfactual import get_counterfactuals
from src.model import train_model

import joblib
import numpy as np
import pandas as pd
import torch

import warnings
warnings.filterwarnings('ignore')

In [2]:
model, encoder, scaler = train_model(device = 'cpu', dataset = 'insurance', n_epoch = 10, \
                                     batch_size_train = 64, learning_rate = .1)


Test set: Avg. loss: 44.5552, Accuracy: 0/134(0%)

Train Epoch: 1 [0/1204 (0%)]	Loss: 0.317271

Test set: Avg. loss: 12.1225, Accuracy: 0/134(0%)

Train Epoch: 2 [0/1204 (0%)]	Loss: 0.068410

Test set: Avg. loss: 13.1770, Accuracy: 0/134(0%)

Train Epoch: 3 [0/1204 (0%)]	Loss: 0.095431

Test set: Avg. loss: 12.1355, Accuracy: 0/134(0%)

Train Epoch: 4 [0/1204 (0%)]	Loss: 0.080463

Test set: Avg. loss: 12.1434, Accuracy: 0/134(0%)

Train Epoch: 5 [0/1204 (0%)]	Loss: 0.081439

Test set: Avg. loss: 12.1079, Accuracy: 0/134(0%)

Train Epoch: 6 [0/1204 (0%)]	Loss: 0.069324

Test set: Avg. loss: 12.1549, Accuracy: 0/134(0%)

Train Epoch: 7 [0/1204 (0%)]	Loss: 0.082426

Test set: Avg. loss: 12.1438, Accuracy: 0/134(0%)

Train Epoch: 8 [0/1204 (0%)]	Loss: 0.092547

Test set: Avg. loss: 12.1243, Accuracy: 0/134(0%)

Train Epoch: 9 [0/1204 (0%)]	Loss: 0.081346

Test set: Avg. loss: 12.3783, Accuracy: 0/134(0%)

Train Epoch: 10 [0/1204 (0%)]	Loss: 0.082243

Test set: Avg. loss: 12.1100, Accuracy

model, encoder, scaler = joblib.load('models/hospital.gz')

In [5]:
loader = iter(load_insurance(100, train=False))
X_corpus, _ = next(loader)
X_test, _ = next(loader)

x = X_test[0:1]
desired_class = f'>{float(model(x))}'
desired_class

'>0.09211967885494232'

In [6]:
cat_indices = list(range(len(encoder.cols)))
cat_indices

[0, 1, 2, 3]

In [7]:
%%time
cfs = get_counterfactuals(model = model , corpus = X_corpus, test = x, \
                        target = desired_class, n_counterfactuals = 10,  
                          cat_indices = cat_indices)

TypeError: get_counterfactuals() missing 1 required positional argument: 'test_latent_approx'

In [ ]:
cols = ['Gender', 'Neighbourhood', 'Scholarship', 'Hipertension', 'Diabetes', \
       'Alcoholism', 'SMS_received', 'Handcap', 'Age', 'ScheduleDays']

def transform(inp):
    return torch.Tensor(scaler.transform(encoder.transform(inp)))

def inverse_transform(inp, columns = cols, round_ = True):
    inp = scaler.inverse_transform(inp.detach().numpy())
    if round_:
        inp = np.round(inp,0).astype(int)
    return encoder.inverse_transform(pd.DataFrame(inp, columns = columns))

In [ ]:
def color_boolean(val):
    return f'background-color: {"green" if val else ""}'

In [ ]:
print('Original: ')
display(inverse_transform(x))
print()

if cfs is None:
    print('No counterfactuals found!')
else:
    for kept_cf in cfs:
        kept_cf = kept_cf.reshape(1,*kept_cf.shape)

        diff = (inverse_transform(x) != inverse_transform(kept_cf))
        #print(diff)
        #print()

        print('Kept counterfactual generation: ')
        display(inverse_transform(kept_cf).style.apply(lambda _: diff.applymap(color_boolean), axis=None))
        print()

        # print('Sparsity: ', int(diff.values.sum()))
        # print()

        print('Predicted: ', model(torch.Tensor(kept_cf)).argmax(), ' || ', 'Desired: ', desired_class, ' || ', 'Orginal: ',  model(x).argmax())

        print('*'*120)